In [ ]:
%matplotlib inline
%pylab inline

import menpo.io as mio
from menpo.feature import no_op, fast_dsift
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images
from menpo.image import Image

## Load images

In [ ]:
images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/trainset/', 
                           verbose=True, max_images=100):
    if i.n_channels == 3:
        i = i.crop_to_landmarks_proportion(0.8)
        i = i.rescale_landmarks_to_diagonal_range(200)
        labeller(i, 'PTS', ibug_face_66)
        images.append(i)

In [ ]:
visualize_images(images)

In [ ]:
shapes = [i.landmarks['ibug_face_66'].lms for i in images]

In [ ]:
from menpo.shape import PointCloud

landmarks = [PointCloud(s.points[40:42,:])  for s in shapes]

In [ ]:
from menpofit.clm.patch_experts import CorrelationFilterExpertEnsemble
from menpo.shape import PointCloud

cfee = CorrelationFilterExpertEnsemble(
    images, 
    landmarks, 
    verbose=True,
    patch_size=(17, 17),
    context_size=(34, 34),
    response_covariance=1,
    sample_offsets=PointCloud([[0,0],[8,0],[0,8],[8,8],[-8,0],[0,-8],[-8,-8]]))

In [ ]:
visualize_images(cfee.spatial_filter_images)

In [ ]:
responses = []
for (image, landmark) in zip(images, landmarks):
    response = Image(cfee.predict(image, landmark)[1])
    responses.append(response)
    
visualize_images(responses)